In [1]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
from fastlane_bot import Bot, Config
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.helpers.poolandtokens import PoolAndTokens
from fastlane_bot.helpers import TradeInstruction, TxReceiptHandler, TxRouteHandler, TxSubmitHandler, TxHelpers, TxHelper
from fastlane_bot.events.managers.manager import Manager
from fastlane_bot.events.interface import QueryInterface
import pytest
import math
import json
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
from fastlane_bot.modes import triangle_single_bancor3
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v2.10.1 (07/May/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.1 (2023-07-03)
UniswapV3 v0.0.1 (2023-07-03)
SushiswapV2 v0.0.1 (2023-07-03)
CarbonV1 v0.0.1 (2023-07-03)
BancorV3 v0.0.1 (2023-07-03)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require


C:\Users\Kveen\AppData\Local\Temp\ipykernel_33356\3226121854.py:27: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.2 [requirements >= 3.0 is met]


In [2]:
C = cfg = Config.new(config=Config.CONFIG_MAINNET)
assert (C.NETWORK == C.NETWORK_MAINNET)
assert (C.PROVIDER == C.PROVIDER_ALCHEMY)
setup_bot = CarbonBot(ConfigObj=C)

In [3]:


pools = None
with open('latest_pool_data_bancor_v3_testing.json') as f:
    pools = json.load(f)
pools = [pool for pool in pools]

In [4]:
pools[0]
static_pools = pools

In [5]:
state = pools
exchanges = list({ex['exchange_name'] for ex in state})
db = QueryInterface(state=state, ConfigObj=C, exchanges=exchanges)
setup_bot.db = db

In [6]:
static_pool_data_filename = "static_pool_data"

static_pool_data = pd.read_csv(f"fastlane_bot/data/{static_pool_data_filename}.csv", low_memory=False)
    
uniswap_v2_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v2_event_mappings.csv", low_memory=False)
        
tokens = pd.read_csv("fastlane_bot/data/tokens.csv", low_memory=False)
        
exchanges = "carbon_v1,bancor_v3,uniswap_v3,uniswap_v2,sushiswap_v2"

exchanges = exchanges.split(",")
arb_mode = "bancor_v3"

alchemy_max_block_fetch = 20

In [7]:
static_pool_data["cid"] = [
        cfg.w3.keccak(text=f"{row['descr']}").hex()
        for index, row in static_pool_data.iterrows()
    ]


In [8]:
# Filter out pools that are not in the supported exchanges
static_pool_data = [
    row for index, row in static_pool_data.iterrows()
    if row["exchange_name"] in exchanges
]

static_pool_data = pd.DataFrame(static_pool_data)

In [9]:
static_pool_data['exchange_name'].unique()

array(['uniswap_v3', 'bancor_v3', 'uniswap_v2', 'sushiswap_v2'],
      dtype=object)

In [10]:
from joblib import Parallel, delayed

# Initialize data fetch manager
mgr = Manager(
    web3=cfg.w3,
    cfg=cfg,
    pool_data=static_pool_data.to_dict(orient="records"),
    SUPPORTED_EXCHANGES=exchanges,
    alchemy_max_block_fetch=alchemy_max_block_fetch,
    uniswap_v2_event_mappings=uniswap_v2_event_mappings,
    tokens=tokens.to_dict(orient="records"),
)

# Add initial pools for each row in the static_pool_data
start_time = time.time()
Parallel(n_jobs=-1, backend="threading")(
    delayed(mgr.add_pool_to_exchange)(row) for row in mgr.pool_data
)
cfg.logger.info(f"Time taken to add initial pools: {time.time() - start_time}")

# check if any duplicate cid's exist in the pool data
mgr.deduplicate_pool_data()
cids = [pool["cid"] for pool in mgr.pool_data]
assert len(cids) == len(set(cids)), "duplicate cid's exist in the pool data"

2023-07-13 15:44:24,862 [fastlane:INFO] - Time taken to add initial pools: 0.09552431106567383


In [11]:
def init_bot(mgr: Manager) -> CarbonBot:
    """
    Initializes the bot.

    Parameters
    ----------
    mgr : Manager
        The manager object.

    Returns
    -------
    CarbonBot
        The bot object.
    """
    mgr.cfg.logger.info("Initializing the bot...")
    bot = CarbonBot(ConfigObj=mgr.cfg)
    bot.db = db
    bot.db.mgr = mgr
    assert isinstance(
        bot.db, QueryInterface
    ), "QueryInterface not initialized correctly"
    return bot

In [12]:
bot = init_bot(mgr)

2023-07-13 15:44:24,902 [fastlane:INFO] - Initializing the bot...


In [13]:
# add data cleanup steps from main.py
bot.db.handle_token_key_cleanup()
bot.db.remove_unmapped_uniswap_v2_pools()
bot.db.remove_zero_liquidity_pools()
bot.db.remove_unsupported_exchanges()

2023-07-13 15:44:25,079 [fastlane:INFO] - Removed 3242 unmapped uniswap_v2/sushi pools. 1897 uniswap_v2/sushi pools remaining
2023-07-13 15:44:25,080 [fastlane:INFO] - Unmapped uniswap_v2/sushi pools:
2023-07-13 15:44:25,248 [fastlane:INFO] - uniswap_v3: 0
2023-07-13 15:44:25,249 [fastlane:INFO] - uniswap_v2: 3242
2023-07-13 15:44:25,250 [fastlane:INFO] - sushiswap_v2: 0
2023-07-13 15:44:25,252 [fastlane:INFO] - uniswap_v3: 636
2023-07-13 15:44:25,254 [fastlane:INFO] - sushiswap_v2: 78
2023-07-13 15:44:25,255 [fastlane:INFO] - uniswap_v2: 0
2023-07-13 15:44:25,256 [fastlane:INFO] - bancor_v2: 0
2023-07-13 15:44:25,257 [fastlane:INFO] - bancor_v3: 34
2023-07-13 15:44:25,257 [fastlane:INFO] - carbon_v1: 220
2023-07-13 15:44:25,280 [fastlane:INFO] - uniswap_v3_zero_liquidity_pools: 858
2023-07-13 15:44:25,281 [fastlane:INFO] - sushiswap_v2_zero_liquidity_pools: 34
2023-07-13 15:44:25,281 [fastlane:INFO] - uniswap_v2_zero_liquidity_pools: 0
2023-07-13 15:44:25,283 [fastlane:INFO] - bancor_

In [14]:
tokens = bot.db.get_tokens()

In [15]:
ADDRDEC = {t.key: (t.address, int(t.decimals)) for t in tokens if not math.isnan(t.decimals)}

In [16]:
flashloan_tokens = bot.setup_flashloan_tokens(["BNT-FF1C"])
CCm = bot.setup_CCm(None)

In [17]:
pools = db.get_pool_data_with_tokens()

In [18]:
single = bot._run(flashloan_tokens=flashloan_tokens, CCm=CCm, arb_mode=arb_mode, data_validator=False, result="calc_trade_instr")

2023-07-13 15:44:25,416 [fastlane:WARNING] - base_exchange must be bancor_v3 for single_triangle_bancor3, setting it to bancor_v3
2023-07-13 15:44:25,417 [fastlane:INFO] - flashloan_tokens for arb_mode=single_triangle_bancor3 will be overwritten. 
c:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: inv

In [19]:
arb_finder = bot._get_arb_finder("bancor_v3")

In [20]:
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=False,
            ConfigObj=bot.ConfigObj,
        )

In [21]:
r = finder.find_arbitrage()

2023-07-13 15:44:26,071 [fastlane:WARNING] - base_exchange must be bancor_v3 for single_triangle_bancor3, setting it to bancor_v3
2023-07-13 15:44:26,071 [fastlane:INFO] - flashloan_tokens for arb_mode=single_triangle_bancor3 will be overwritten. 
c:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: invalid value encountered in subtract
  jac[:, j] = (func(x_plus) - y) / Dxj
c:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
c:\Users\Kveen\PycharmProjects\carbonbot\resources\NBTest\fastlane_bot\tools\optimizer.py:1157: RuntimeWarning: inv

In [22]:
print(len(r))

5


In [23]:
(
            best_profit,
            best_trade_instructions_df,
            best_trade_instructions_dic,
            best_src_token,
            best_trade_instructions,
        ) = r

In [25]:
(
ordered_trade_instructions_dct,
tx_in_count,
) = bot._simple_ordering_by_src_token(
best_trade_instructions_dic, best_src_token
)

In [26]:
pool_cids = [curve['cid'] for curve in ordered_trade_instructions_dct]

In [27]:
first_check_pools = finder.get_exact_pools(pool_cids)

In [28]:
first_check_pools

[ConstantProductCurve(k=14219223533092.309, x=14945274.66925332, x_act=14945274.66925332, y_act=951419.351451954, pair='BNT-FF1C/LINK-86CA', cid='0x221a61aaf35ba8ca83dc227742b7e1ab7ea9ca75a4db843b27da6c367d694311', fee=0.0, descr='bancor_v3 BNT-FF1C/LINK-86CA 0.000', constr='uv2', params={'exchange': 'bancor_v3', 'tknx_dec': 18, 'tkny_dec': 18, 'tknx_addr': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'tkny_addr': '0x514910771AF9Ca656af840dff83E8264EcF986CA', 'blocklud': 17674416}),
 ConstantProductCurve(k=679511874103.592, x=330089.83031239477, x_act=315.2013455032138, y_act=4204.436000232352, pair='LINK-86CA/USDC-eB48', cid='0x2f7a9821444c1a975631ac55a1aee5360da436100cc9af2e5010397eb78a6b7c', fee=0.003, descr='uniswap_v3 LINK-86CA/USDC-eB48 3000', constr='pkpp', params={'exchange': 'uniswap_v3', 'tknx_dec': 18, 'tkny_dec': 6, 'tknx_addr': '0x514910771AF9Ca656af840dff83E8264EcF986CA', 'tkny_addr': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', 'blocklud': 17675270, 'L': 824325.1021918

In [29]:
assert(len(first_check_pools) == 3), f"[test_bancor_v3] Validation expected 3 pools, got {len(first_check_pools)}"
for pool in first_check_pools:
    assert type(pool) == ConstantProductCurve, f"[test_bancor_v3] Validation pool type mismatch, got {type(pool)} expected ConstantProductCurve"
    assert pool.cid in pool_cids, f"[test_bancor_v3] Validation missing pool.cid {pool.cid} in {pool_cids}"


In [30]:
optimal_arb = finder.get_optimal_arb_trade_amts(pool_cids, flashloan_tokens[0])
assert type(optimal_arb) == float, f"[test_bancor_v3] Optimal arb calculation type is {type(optimal_arb)} not float"
assert iseq(optimal_arb, 5003.2368760578265), f"[test_bancor_v3] Optimal arb calculation type is {optimal_arb}, expected 5003.2368760578265"


5003.2368760578265


In [31]:
flt='BNT-FF1C'

tkn0 = finder.get_tkn(pool=first_check_pools[0], tkn_num=0)
tkn1 = finder.get_tkn(pool=first_check_pools[0], tkn_num=1)
tkn2 = finder.get_tkn(pool=first_check_pools[1], tkn_num=0)
tkn5 = finder.get_tkn(pool=first_check_pools[2], tkn_num=0)
p0t0 = first_check_pools[0].x_act if tkn0 == flt else first_check_pools[0].y_act
p0t1 = first_check_pools[0].y_act if tkn0 == flt else first_check_pools[0].x_act
p2t1 = first_check_pools[2].x_act if tkn5 == flt else first_check_pools[2].y_act
p2t0 = first_check_pools[2].y_act if tkn5 == flt else first_check_pools[2].x_act
p1t0 = first_check_pools[1].x if tkn1 == tkn2 else first_check_pools[1].y
p1t1 = first_check_pools[1].y if tkn1 == tkn2 else first_check_pools[1].x
fee1 = finder.get_fee_safe(first_check_pools[1].fee)

In [32]:
optimal_arb_low_level_check = finder.max_arb_trade_in_constant_product(p0t0=p0t0, p0t1=p0t1, p1t0=p1t0, p1t1=p1t1, p2t0=p2t0, p2t1=p2t1,fee0=0, fee1=fee1, fee2=0)
assert iseq(optimal_arb, optimal_arb_low_level_check), f"[test_bancor_v3] Arb calculation result mismatch, pools likely ordered incorrectly"

In [33]:
ext_fee = finder.get_fee_safe(first_check_pools[1].fee)
assert type(ext_fee) == float, f"[test_bancor_v3] Testing external pool, fee type is {type(ext_fee)} not float"
assert iseq(ext_fee, 0.003), f"[test_bancor_v3] Testing external pool, fee amt is {ext_fee} not 0.003"

In [34]:
flt = {'MKR-79A2', 'TRAC-0A6F', 'MONA-412A', 'WBTC-C599', 'WOO-5D4B', 'MATIC-eBB0', 'BAT-87EF', 'UOS-5C8c', 'LRC-EafD', 'NMR-6671', 'DIP-cD83', 'TEMP-1aB9', 'ICHI-A881', 'USDC-eB48', 'ENS-9D72', 'vBNT-7f94', 'ANKR-EDD4', 'UNI-F984', 'REQ-938a', 'WETH-6Cc2', 'AAVE-DaE9', 'ENJ-3B9c', 'MANA-C942', 'wNXM-2bDE', 'QNT-4675', 'RLC-7375', 'CROWN-E0fa', 'CHZ-b4AF', 'USDT-1ec7', 'DAI-1d0F', 'RPL-A51f', 'HOT-26E2', 'LINK-86CA', 'wstETH-2Ca0'}

In [35]:
combos = finder.get_combos(flashloan_tokens=flt, CCm=CCm, arb_mode="bancor_v3")

In [36]:
assert len(combos) == 1122, "[test_bancor_v3] Different data used for tests, expected 1122 combos"

In [37]:
all_miniverses = finder.get_miniverse_combos(combos)

In [38]:
assert len(all_miniverses) == 146, "[test_bancor_v3] Different data used for tests, expected 146 miniverses"

In [39]:
bancor_v3_curve_0 = (
                finder.CCm.bypairs(f"BNT-FF1C/WETH-6Cc2")
                .byparams(exchange="bancor_v3")
                .curves
            )
bancor_v3_curve_1 = (
                finder.CCm.bypairs(f"BNT-FF1C/USDC-eB48")
                .byparams(exchange="bancor_v3")
                .curves
            )

In [40]:
carbon_curves = finder.CCm.bypairs(f"USDC-eB48/WETH-6Cc2")
carbon_curves = list(set(carbon_curves))
carbon_curves = [
    curve
    for curve in carbon_curves
    if curve.params.get("exchange") == "carbon_v1"
]

In [41]:
miniverse = [bancor_v3_curve_0 + bancor_v3_curve_1 + carbon_curves]

In [42]:
max_arb_carbon = finder.run_main_flow(miniverse=miniverse[0], src_token="BNT-FF1C")

In [43]:
(
profit_src_0,
trade_instructions_0,
trade_instructions_df_0,
trade_instructions_dic_0,
) = max_arb_carbon

In [44]:
mono_carbon = finder.get_mono_direction_carbon_curves(miniverse[0], trade_instructions_df=trade_instructions_df_0, token_in=None)

In [45]:
test_mono_carbon = finder.get_mono_direction_carbon_curves(miniverse[0], trade_instructions_df=trade_instructions_df_0, token_in='WETH-6Cc2')

In [46]:
# Test that get_mono_direction_carbon_curves removed two curves
assert len(test_mono_carbon) != len(mono_carbon), f"[test_bancor_v3] Issue with get_mono_direction_carbon_curves, should have removed one or more pools"